## Google API Table Pivot

In [385]:
# Reads a google sheet table, processes it and keeps an updated version on another sheet.
!which python

/home/eduardo/anaconda3/bin/python


In [386]:
!pip install oauthlib
!pip3 install gspread
!pip3 install --upgrade google-api-python-client oauth2client 

In [995]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

import time
import ipywidgets as widgets

import json

In [996]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1DRD97TAw2WIuTCG0Nh6BW-aVvDKAgY1wvJb38V-3vU8'
SAMPLE_RANGE_NAME = 'Reto1!A2:E'

TEST_SPREADSHEET_ID='1aqM5_3d8Wrw522LF5scS2h28hCvNSF8FO2C8nOe4Ypc'

index_columns=['Author','Sentiment', 'Country','Theme']

In [1010]:
# Use main to use datasheet in owner drive
def main():
    """Shows basic usage of the Sheets API.
    Returns values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_264260318778-6q86htiapp1m19qc88ksmddpfnfdjkvi.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    #if not values:
    #    print('No data found.')
    #else:
    #    print('Author, Sentiment, Country, Theme:')
    #    for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
    #        print(f'{row[0]}, {row[1]}, {row[2]}, {row[3]}')
    return values

# Use main_test to use datasheet in tester drive
def main_test():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_264260318778-6q86htiapp1m19qc88ksmddpfnfdjkvi.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=TEST_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    #if not values:
    #    print('No data found.')
    #else:
    #    print('Author, Sentiment, Country, Theme:')
    #    for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
    #        print(f'{row[0]}, {row[1]}, {row[2]}, {row[3]}')
    return values


In [1013]:
#Production (challenge)
table_values = main_test()

#Development
#table_values = main()

In [1014]:
df = pd.DataFrame(table_values, columns=index_columns)

df_c = pd.DataFrame(table_values, columns=index_columns)
df_t = pd.DataFrame(table_values, columns=index_columns)
df

,Author,Sentiment,Country,Theme
0,Joseluna,Positive,Ecuador,PA
1,sixt93,Negative,Colombia,CA
2,alberto,Positive,Francia,OT
3,carlosz,Negative,USA,OT
4,jgbesq,Negative,UK,PA
5,nuri90,Neutral,Spain,EV
6,santiago8,Neutral,Chile,KS
7,Joseluna,Positive,Peru,KS
8,alberto,Positive,Francia,OT
9,jgbesq,Negative,Panama,KS


In [1004]:
pivot1 = df.pivot_table(index=['Author','Sentiment'], columns=['Country'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE')
pivot2 = df.pivot_table(index=['Author','Sentiment'], columns=['Theme'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE')
all_data = pd.merge(pivot1, pivot2, 'left', on = ['Author','Sentiment'] )
all_data

Theme                                                \
Country             Bolivia  Chile Colombia Ecuador Francia Panama   Peru   
Author    Sentiment                                                         
Joseluna  Positive    FALSE  FALSE    FALSE    TRUE   FALSE  FALSE   TRUE   
alberto   Positive    FALSE  FALSE    FALSE   FALSE    TRUE  FALSE  FALSE   
carlosz   Negative    FALSE  FALSE    FALSE   FALSE   FALSE  FALSE  FALSE   
jgbesq    Negative    FALSE  FALSE    FALSE   FALSE   FALSE   TRUE  FALSE   
nico23    Neutral     FALSE  FALSE    FALSE   FALSE   FALSE  FALSE  FALSE   
nuri90    Neutral     FALSE  FALSE    FALSE   FALSE   FALSE  FALSE  FALSE   
santiago8 Neutral      TRUE   TRUE    FALSE   FALSE   FALSE  FALSE  FALSE   
sixt93    Negative    FALSE  FALSE     TRUE   FALSE   FALSE  FALSE  FALSE   

                                                  Country                \
Country             Portugal  Spain     UK    USA      CA     EV     KS   
Author    Sentiment                                                       
Joseluna  Positive     FALSE  FALSE  FALSE  FALSE   FALSE  FALSE   TRUE   
alberto   Positive     FALSE  FALSE  FALSE   TRUE   FALSE  FALSE   TRUE   
carlosz   Negative     FALSE  FALSE  FALSE   TRUE   FALSE  FALSE  FALSE   
jgbesq    Negative     FALSE  FALSE   TRUE  FALSE   FALSE  FALSE   TRUE   
nico23    Neutral      FALSE  FALSE  FALSE   TRUE   FALSE  FALSE  FALSE   
nuri90    Neutral      FALSE   TRUE  FALSE  FALSE   FALSE   TRUE  FALSE   
santiago8 Neutral      FALSE  FALSE  FALSE  FALSE   FALSE  FALSE   TRUE   
sixt93    Negative      TRUE  FALSE  FALSE  FALSE    TRUE  FALSE  FALSE   

                                   
Country                 OT     PA  
Author    Sentiment                
Joseluna  Positive   FALSE   TRUE  
alberto   Positive    TRUE  FALSE  
carlosz   Negative    TRUE  FALSE  
jgbesq    Negative   FALSE   TRUE  
nico23    Neutral    FALSE   TRUE  
nuri90    Neutral    FALSE  FALSE  
santiago8 Neutral    FALSE   TRUE  
sixt93    Negative    TRUE  FALSE

In [1005]:
# Use this block only to check that the information is being updated, stop and and run following blocks to continue
out = widgets.HTML()
display(out)
while True:
    time.sleep(1)
    table_values = main_test()
    df = pd.DataFrame(table_values, columns=index_columns)
    pivot1 = df.pivot_table(index=['Author','Sentiment'], columns=['Country'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE')
    pivot2 = df.pivot_table(index=['Author','Sentiment'], columns=['Theme'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE')
    all_data = pd.merge(pivot1, pivot2, 'left', on = ['Author','Sentiment'] )
    
    out.value = all_data.to_html().replace("\n",'\\').replace("\\",'') 

HTML(value='')

KeyboardInterrupt: 

In [1006]:
import pickle

In [1007]:
#change this by your sheet ID
SAMPLE_SPREADSHEET_ID_input = TEST_SPREADSHEET_ID
SCOPES_2 = ['https://www.googleapis.com/auth/spreadsheets']
SAMPLE_RANGE_NAME_w = 'ejemplo_reto1!A2:AA1000'

def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
    global service
    SCOPES = [scope for scope in scopes[0]]
    #print(SCOPES)
    
    cred = None

    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
            cred = flow.run_local_server()

        with open('token_write.pickle', 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(api_service_name, api_version, credentials=cred)
        print(api_service_name, 'service created successfully')
        #return service
    except Exception as e:
        print(e)
        #return None

In [1008]:
# change 'my_json_file.json' by your downloaded JSON file.
Create_Service('client_secret_264260318778-6q86htiapp1m19qc88ksmddpfnfdjkvi.apps.googleusercontent.com.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

sheets service created successfully


In [1016]:
# Updates to google drive
out = widgets.HTML()
display(out)
while True:
    time.sleep(2)
    table_values = main_test()
    
    df = pd.DataFrame(table_values, columns=index_columns)
    df_c = pd.DataFrame(table_values, columns=index_columns)
    df_t = pd.DataFrame(table_values, columns=index_columns)

    df_t.pop('Country')
    df_c.pop('Theme')

    pivot_test_c = df_c.pivot_table(index=['Author','Sentiment'], columns=['Country'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE').reset_index()


    pivot_test_t = df_t.pivot_table(index=['Author','Sentiment'], columns=['Theme'], values=['Sentiment'],
                        aggfunc=lambda x: 'TRUE', fill_value = 'FALSE').reset_index()

    pivot_test_all_data = pd.merge(pivot_test_c, pivot_test_t, 'left', on = ['Author','Sentiment'] )
    
    
    out.value = pivot_test_all_data.to_html().replace("\n",'\\').replace("\\",'')

    response_date = service.spreadsheets().values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
        valueInputOption='RAW',
        range=SAMPLE_RANGE_NAME_w,
        body=dict(
            majorDimension='ROWS',
            values=pivot_test_all_data.T.reset_index().T.values.tolist())
    ).execute()
    #print('Sheet successfully Updated')
    

HTML(value='')

KeyboardInterrupt: 